In [63]:
#request & response handler
import urllib.request

def start_request(self):
    url = "https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407"
    response = urllib.request.urlopen(url)
    yield Request(url, callback=self.parse)
    #print (response.read())


In [64]:
# parser
import urllib
import requests
from lxml import html
from lxml import etree
import json

def parse(self, response):
    page = requests.get('https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407')
    tree = etree.HTML(page.content)

    delivery_history = tree.xpath('//div')
    #print(delivery_history)

    string = str(etree.tostring(delivery_history[0]))
    start = string.find('JSON.parse(')
    end = string.find('"),', start)
    json_string = string[start:end]
    json_string = json_string.replace('JSON.parse("', '')
    json_string = json_string.replace('\\', '')
    shipment_details = json.loads(json_string)
    print(shipment_details)




In [65]:
import datetime

delivery_history_list = []
delivery_history_dict = {}
    
delivery_history_dict["shipment_number"]=shipment_details["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
delivery_history_dict["crawltime"]=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
delivery_history_dict["url"]=url
delivery_history_dict["events"]=shipment_details["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])

delivery_history_list.append(delivery_history_dict)
print(delivery_history_dict)

{'shipment_number': '00340434188193324407', 'crawltime': '2019-10-15 16:08:14', 'url': (((('https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407',),),),), 'events': [{'datum': '2019-10-07T16:27:00+02:00', 'status': 'Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.'}, {'datum': '2019-10-08T17:12:00+02:00', 'ort': 'Bremen', 'status': 'Die Sendung wurde von DHL bearbeitet und wird fu00FCr den Weitertransport in die Region des Empfu00E4ngers vorbereitet.'}, {'datum': '2019-10-09T10:25:00+02:00', 'ort': 'Neuwied', 'status': 'Die Sendung ist in der Region des Empfu00E4ngers angekommen und wird im nu00E4chsten Schritt zur Zustellbasis transportiert.'}, {'datum': '2019-10-10T05:58:00+02:00', 'ort': 'Bonn MechZB', 'status': 'Die Sendung wird fu00FCr die Verladung ins Zustellfahrzeug vorbereitet.'}, {'datum': '2019-10-10T09:19:00+02:00', 'status': 'Die Sendung wurde in 

In [66]:
import pandas

#make a dataframe with the list
df=pandas.DataFrame(delivery_history_list)

#write the df to a csv file
df.to_csv("delivery_history.csv")